# Oscilloscope synchronized with generator

This example shows how to simultaneously generate and acquired with generated a signal.

**Note:**  
Voltage range of fast analog inputs on the Red Pitaya depends on gain setting that can be set by jumpers. HV setting is for input range to ±20V, while LV sets input range to ±1V. For more information please read the following [chapter](http://redpitaya.readthedocs.io/en/latest/doc/developerGuide/125-14/fastIO.html#analog-inputs). 

Create a loop back from fast outputs to the fast inputs as shown in the picture below.  
Make sure that *gain setting* is set to ±1V (LV).

![Fast loop back](img/FastIOLoopBack.png "Example of the fast loop back.")

Device tree overlay

In [ ]:
from redpitaya.overlay.mercury import mercury as overlay
fpga = overlay()

gen0 = fpga.gen(0)
osc0 = fpga.osc(0, 1.0)

Arbitrary signal generator

In [ ]:
# buffer waveform and sample timing
gen0.waveform = gen0.sin()

# burst half the buffer with then idle for quarter buffer, repeat 4 times
gen0.mode                   = 'BURST'
gen0.burst_data_repetitions = 1
gen0.burst_data_length      =  gen0.buffer_size // 2
gen0.burst_period_length    = (gen0.buffer_size // 4) * 3
gen0.burst_period_number    = 4

# set output amplitude, offset and enable it
gen0.amplitude     = 1.0
gen0.offset        = 0
gen0.enable        = True

Acquire (oscilloscope)

In [ ]:
# data rate decimation 
osc0.decimation = 4

# trigger timing
N = osc0.buffer_size
osc0.trigger_pre  = 0
osc0.trigger_post = N

# synchronization signals from 
osc0.sync_src = fpga.sync_src['gen0']
osc0.trig_src = 0

Both the generator and oscilloscope are started at the same time.

In [ ]:
# reset, start and trigger generator to get the first burst
gen0.reset()
#gen0.start()
#gen0.trigger()
gen0.start_trigger()
while (osc0.status_run()): pass

In [ ]:
import matplotlib.pyplot as plt

# show only the part of the buffer requested by pre/post trigger timing
data = osc0.data(N)
plt.plot(data)
plt.show()